## Matrix and Vocabulary Construction

In [43]:
import pandas as pd
import operator
import numpy as np
from scipy import sparse
import nltk
from nltk import bigrams    
import scipy.sparse as sps

In [44]:
news = pd.read_csv("./estadao_noticias_eleicao.csv", encoding="utf-8")

In [45]:
content = news.titulo + " " + news.conteudo
content = content.fillna("")

In [46]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

In [47]:
tokens_lists = content.apply(lambda text: text.lower().split())

In [48]:
tokens = [token for tokens_list in tokens_lists for token in tokens_list]

In [49]:
matrix, vocab = co_occurrence_matrix(tokens)

## Consult Bigram Frequency

In [50]:
consultable_matrix = matrix.tocsr()

In [51]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

### Example

In [52]:
w1 = 'poucos'
w2 = 'recursos'
consult_frequency(w1, w2)

3

## Top-3 em ordem decrescente

In [108]:
def top3(w1):
    top3 = {}
    for w in consultable_matrix[vocab[w1]].indices:
        if(len(top3) < 3):
            top3[w] = consultable_matrix[vocab[w1], w]
        else:
            if(consultable_matrix[vocab[w1], w] > consultable_matrix[vocab[w1], min(top3, key=top3.get)]):
                del top3[min(top3, key=top3.get)]
                top3[w] = consultable_matrix[vocab[w1], w]
    
    words = [str(vocab.keys()[vocab.values().index(top3.keys()[0])]), str(vocab.keys()[vocab.values().index(top3.keys()[1])]), str(vocab.keys()[vocab.values().index(top3.keys()[2])])]
    return words
                

In [109]:
print(top3("segundo"))

['o', 'a', 'ele,']
